### Reading the data

In [1]:
import sqlite3
import pandas as pd
import zipfile
import io

# Connect to the SQLite database
conn = sqlite3.connect(r"C:\Users\91998\OneDrive\Desktop\Internship_folder\Search_Engine\data\eng_subtitles_database.db")

# Query to fetch data from the 'zipfiles' table
query = "SELECT num, name, content FROM zipfiles"

# Fetch data from the 'zipfiles' table
data = conn.execute(query).fetchall()

# Close the database connection
conn.close()

# Decode the content and store it in a DataFrame
subtitles_list = []
errors = []

for num, name, content in data:
    try:
        # Try to decompress as ZIP format
        with zipfile.ZipFile(io.BytesIO(content)) as z:
            # Assuming there's only one file in the ZIP archive
            file_name = z.namelist()[0]
            decoded_content = z.read(file_name).decode('latin-1')
    except zipfile.BadZipFile:
        try:
            # Try decoding directly as 'latin-1' encoded string
            decoded_content = content.decode('latin-1')
        except Exception as e:
            errors.append((name, str(e)))
            print(f"Error decoding subtitle {name}: {e}")
            continue
    except Exception as e:
        errors.append((name, str(e)))
        print(f"Error processing subtitle {name}: {e}")
        continue
    
    subtitles_list.append([num, name, decoded_content])

# Create a DataFrame from the decoded subtitles list
df = pd.DataFrame(subtitles_list, columns=['num', 'name', 'content'])

# Display the DataFrame
print(df.head())

# Display the errors
if errors:
    print("\nErrors:")
    for name, error in errors:
        print(f"{name}: {error}")


       num                                               name  \
0  9180533                         the.message.(1976).eng.1cd   
1  9180583  here.comes.the.grump.s01.e09.joltin.jack.in.bo...   
2  9180592    yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd   
3  9180594    yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd   
4  9180600                              broker.(2022).eng.1cd   

                                             content  
0  1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an...  
1  1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther...  
2  1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'...  
3  1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an...  
4  ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch...  


## Read the data from csv file

### Removing time stamp deom the content

In [2]:
import pandas as pd
subtitle_df=pd.read_csv(r"C:\Users\91998\OneDrive\Desktop\Internship_folder\Search_Engine\csv_files\subtitle_data.csv")
subtitle_df.head()

,num,name,content
0,9180533,the.message.(1976).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,9180600,broker.(2022).eng.1cd,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."


In [3]:
import re
# Regular expression pattern to remove timestamps
timestamp_pattern = r'\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}'
# Regular expression pattern to remove unwanted characters like '\r', '\n', 'ï»¿'
unwanted_pattern = r'[\r\nï»¿]'

# Function to clean the content
def clean_content(text):
    text = re.sub(timestamp_pattern, '', text)  # Remove timestamps
    text = re.sub(unwanted_pattern, '', text)    # Remove unwanted characters
    text = re.sub(r'<.*?>', '', text)            # Remove HTML tags
    text = text.strip()                          # Strip leading/trailing whitespace
    text = re.sub(r'^\d+', '', text.strip())     # Remove leading '1'
    return text.strip()

# Apply the cleaning function to the 'content' column
subtitle_df['cleaned_content'] = subtitle_df['content'].apply(clean_content)

# Display the cleaned DataFrame
subtitle_df[['content', 'cleaned_content']]

,content,cleaned_content
0,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an...",Watch any video online with Open-SUBTITLESFree...
1,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther...",Ah! There's PrincessDawn and Terry with the2Bl...
2,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'...",Yumi's Cells 22Episode 36Extremely Polite Yumi...
3,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an...",Watch any video online with Open-SUBTITLESFree...
4,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch...",Watch any video online with Open-SUBTITLESFree...
...,...,...
82493,"ï»¿1\r\n00:01:16,284 --> 00:01:19,537\r\nGod,\...","God,why are you punishing me?2""With red onhis ..."
82494,"1\r\n00:00:06,000 --> 00:00:12,074\r\napi.Open...","api.OpenSubtitles.org is deprecated, pleaseimp..."
82495,"1\r\n00:00:01,001 --> 00:00:04,630\r\n(Dramati...",(Dramatic orchestral music)2Advertise your pro...
82496,"1\r\n00:00:06,000 --> 00:00:12,074\r\nAdvertis...",Advertise your product or brand herecontact ww...


In [4]:
subtitle_df.head()

,num,name,content,cleaned_content
0,9180533,the.message.(1976).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an...",Watch any video online with Open-SUBTITLESFree...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther...",Ah! There's PrincessDawn and Terry with the2Bl...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'...",Yumi's Cells 22Episode 36Extremely Polite Yumi...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an...",Watch any video online with Open-SUBTITLESFree...
4,9180600,broker.(2022).eng.1cd,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch...",Watch any video online with Open-SUBTITLESFree...


In [7]:
## step3: Data Preprocessing on train data
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [8]:
def preprocess(raw_text):
    # Removing special characters and digits
    sentence = re.sub("[^a-zA-Z]", " ", raw_text)
    
    # change sentence to lower case
    sentence = sentence.lower()

    # tokenize into words
    tokens =nltk.word_tokenize(sentence)
    
      # Lemmatize
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # remove stop words                
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in lemmatized_tokens if word.lower() not in stop_words]
    
     # Join and return
    return " ".join(filtered_tokens)

In [9]:
from tqdm import tqdm, tqdm_notebook

In [10]:
tqdm.pandas()

In [11]:
processed_content = subtitle_df["cleaned_content"].progress_apply(lambda x: preprocess(x))

100%|██████████████████████████████████████████████████████████████████████████| 82498/82498 [1:06:29<00:00, 20.68it/s]


In [12]:
#processed_content.to_csv('preprocessed_content.csv', index=False)

In [1]:
import pandas as pd
preprocessed_df=pd.read_csv(r"C:\Users\91998\OneDrive\Desktop\Internship_folder\Search_Engine\csv_files\preprocessed_content.csv")
preprocessed_df.head()

,cleaned_content
0,watch video online open subtitlesfree browser ...
1,ah princessdawn terry blooney looney soldiersp...
2,yumi cell episode extremely polite yumi yumi g...
3,watch video online open subtitlesfree browser ...
4,watch video online open subtitlesfree browser ...


In [2]:
preprocessed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cleaned_content  82497 non-null  object
dtypes: object(1)
memory usage: 644.6+ KB


In [3]:
preprocessed_df = preprocessed_df.dropna(subset=['cleaned_content'])

## BOW

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

# Bag of Words (BOW)
vectorizer_bow = CountVectorizer()
bow_matrix = vectorizer_bow.fit_transform(preprocessed_df['cleaned_content'])

bow_matrix.shape

(82497, 2557036)

## TFIDF

In [19]:
from sklearn.feature_extraction.text import  TfidfVectorizer
# TFIDF
vectorizer_tfidf = TfidfVectorizer()
tfidf_matrix = vectorizer_tfidf.fit_transform(preprocessed_df['cleaned_content'])
tfidf_matrix.shape

(82497, 2557036)

## BERT

In [4]:
from sentence_transformers import SentenceTransformer

# Load SentenceTransformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Generate embeddings
embeddings = model.encode(preprocessed_df['cleaned_content'].tolist(), convert_to_tensor=True)
 

In [5]:
embeddings

tensor([[-0.2353,  0.6819, -0.2076,  ..., -0.1899,  0.3273,  0.2456],
        [-0.2543,  0.4076,  0.3067,  ...,  0.0557, -0.0776,  0.1360],
        [-0.4014,  0.1876,  0.4278,  ...,  0.0759, -0.5714,  0.1475],
        ...,
        [-0.0559,  0.3400,  0.2181,  ..., -0.2866,  0.1645,  0.1213],
        [-0.1985,  0.4563,  0.0225,  ..., -0.6298,  0.1717,  0.0029],
        [-0.1153,  0.1704,  0.1474,  ..., -0.0688, -0.4391, -0.0498]])

In [15]:
embeddings.size()

torch.Size([82497, 384])

In [4]:
import pandas as pd


embeddings_df = pd.DataFrame(embeddings.numpy())

# Save DataFrame to CSV
#embeddings_df.to_csv('embeddings.csv', index=False, header=False)

NameError: name 'embeddings' is not defined

In [5]:
import sqlite3
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
# Create a new SQLite database or connect to an existing one
conn = sqlite3.connect('chroma_db.db')
c = conn.cursor()

# Create table to store embeddings
c.execute('''CREATE TABLE IF NOT EXISTS embeddings
             (video_id TEXT, chunk_id INTEGER, embedding BLOB)''')

conn.commit()


In [7]:
def store_embeddings_in_db(video_id, embeddings):
    """
    Store embeddings in ChromaDB.
    
    Parameters:
    - video_id: ID of the video
    - embeddings: List of embeddings to store
    """
    for i, emb in enumerate(embeddings):
        c.execute("INSERT INTO embeddings (video_id, chunk_id, embedding) VALUES (?, ?, ?)",
                  (video_id, i, sqlite3.Binary(emb)))
    conn.commit()


In [9]:
def retrieve_embeddings_from_db(video_id):
    """
    Retrieve embeddings from ChromaDB.
    
    Parameters:
    - video_id: ID of the video
    
    Returns:
    - List of embeddings
    """
    c.execute("SELECT embedding FROM embeddings WHERE video_id=?", (video_id,))
    rows = c.fetchall()
    embeddings = [np.frombuffer(row[0], dtype=np.float32) for row in rows]
    return embeddings


In [10]:
def preprocess_query(query):
    # Add preprocessing steps here if required
    return query


In [11]:
def create_query_embedding(query):
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # You can use any other SentenceTransformer model
    query_embedding = model.encode([query])
    return query_embedding


In [12]:
def calculate_similarity(video_id, query_embedding):
    # Retrieve embeddings for the specified video from ChromaDB
    video_embeddings = retrieve_embeddings_from_db(video_id)
    
    # Calculate cosine similarity between query embedding and video embeddings
    similarities = [cosine_similarity(query_embedding, emb.reshape(1, -1))[0][0] for emb in video_embeddings]
    
    return similarities


In [13]:
def retrieve_most_relevant_documents(video_id, query, top_n=5):
    preprocessed_query = preprocess_query(query)
    query_embedding = create_query_embedding(preprocessed_query)
    
    similarities = calculate_similarity(video_id, query_embedding)
    
    # Get indices of top N most similar chunks
    top_indices = np.argsort(similarities)[-top_n:][::-1]
    
    # Retrieve the actual subtitles/chunks from the database or file based on the top indices
    relevant_chunks = [f"Chunk {i}: {'Subtitle Text'}" for i in top_indices]  # Placeholder
    
    return relevant_chunks


In [14]:
video_id = "example_video"
query = "your search query here"

# Retrieve most relevant documents
relevant_documents = retrieve_most_relevant_documents(video_id, query)
print(relevant_documents)


[]


In [14]:
embeddings.size()

torch.Size([82497, 384])

In [8]:
vectors=pd.read_csv(r"C:\Users\91998\OneDrive\Desktop\Internship_folder\Search_Engine\embeddings.csv")

In [9]:
vectors.head()

,-0.23528694,0.68194026,-0.20762983,-0.56756777,-0.09497115,0.1533612,0.25356835,-0.3048311,0.3378566,0.24762796,...,0.40654382,-0.18575114,0.28627804,-0.05794515,-0.10239918,0.35828227,0.060026817,-0.18990222,0.32729062,0.24560113
0,-0.254260,0.407602,0.306741,-0.282999,-0.491599,-0.069591,0.993745,-0.291691,-0.190371,-0.142749,...,0.471920,-0.233267,-0.229729,0.169564,-0.588158,0.378377,-0.386424,0.055715,-0.077641,0.135979
1,-0.401395,0.187569,0.427840,0.018657,-0.483465,-0.130123,0.268526,-0.090406,-0.153078,0.078968,...,0.159433,-0.270341,0.266337,0.673169,0.357920,0.453328,-0.068693,0.075851,-0.571419,0.147531
2,-0.519580,-0.098217,0.339848,-0.236897,-0.268159,-0.127692,0.125231,-0.033015,0.043812,-0.034662,...,0.316880,-0.199304,0.212499,0.581604,0.040233,0.661260,0.162934,0.016919,-0.327428,0.115916
3,-0.084290,0.079209,0.479737,-0.283084,-0.049997,-0.041080,0.651616,-0.077211,-0.138057,-0.036446,...,0.397732,-0.323947,0.257993,0.408697,-0.275310,0.456013,-0.264807,0.252055,-0.279540,0.385999
4,-0.041927,0.483662,0.258255,-0.066078,-0.339929,0.014761,0.864491,-0.226859,0.109403,0.262193,...,0.146519,0.418031,0.203830,0.113073,-0.249144,0.501759,-0.291552,-0.212893,-0.306804,0.110402


In [5]:
def chunk_embeddings(embeddings_row, chunk_size=128, overlap=32):
    embeddings = embeddings_row.values
    for start in range(0, len(embeddings), chunk_size - overlap):
        yield embeddings[start:start + chunk_size]

In [6]:
import sqlite3
import pandas as pd
import zipfile
import io
# Connect to SQLite database
conn = sqlite3.connect(r"C:\Users\91998\OneDrive\Desktop\Internship_folder\Search_Engine\vectors.db")
cursor = conn.cursor()

# Create table to store chunks
cursor.execute('''CREATE TABLE IF NOT EXISTS chunks (
                    id INTEGER PRIMARY KEY,
                    document_id INTEGER NOT NULL,
                    chunk_index INTEGER NOT NULL,
                    embedding_chunk TEXT NOT NULL
                )''')

# Function to insert chunks into the SQLite database in batches
def insert_chunks(document_id, chunks):
    cursor.executemany("INSERT INTO chunks (document_id, chunk_index, embedding_chunk) VALUES (?, ?, ?)",
                       [(document_id, i+1, ','.join(map(str, chunk))) for i, chunk in enumerate(chunks)])
    conn.commit()

for idx, row in vectors.iterrows():
    document_id = idx + 1  
    
    # Chunk the embeddings
    chunks = chunk_embeddings(row)
    
    # Insert chunks into SQLite
    insert_chunks(document_id, chunks)

# Close connection to SQLite database
conn.close()

In [10]:
conn = sqlite3.connect(r"C:\Users\91998\OneDrive\Desktop\Internship_folder\Search_Engine\vectors.db")

query = "SELECT * FROM chunks" 
    
# Fetch data into a DataFrame
vec_df = pd.read_sql_query(query, conn)
    
# Display the DataFrame
vec_df.head()

,id,document_id,chunk_index,embedding_chunk
0,1,1,1,"-0.25425997,0.40760165,0.30674076,-0.28299913,..."
1,2,1,2,"0.35028154,-0.11853625,-0.6772548,0.22706099,0..."
2,3,1,3,"-0.15815595,-0.15216613,-0.2958026,0.059707265..."
3,4,1,4,"-0.1082776,0.16064844,-0.11414268,0.40882686,0..."
4,5,2,1,"-0.40139484,0.18756877,0.42783958,0.018657252,..."


In [11]:
vec_df.shape

(329984, 4)

In [12]:
import numpy as np
vec_df = vec_df.drop('id',axis = 1)

In [13]:
# Create a temporary index starting from 1 in original_df
subtitle_df['temp_index'] = np.arange(1, len(subtitle_df) + 1)

# Merge embeddings with original DataFrame based on 'temp_index'
merged_df = pd.merge(vec_df,subtitle_df, left_on='document_id', right_on='temp_index', how='left')
merged_df.head()

,document_id,chunk_index,embedding_chunk,num,name,content,cleaned_content,temp_index
0,1,1,"-0.25425997,0.40760165,0.30674076,-0.28299913,...",9180533,the.message.(1976).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an...",Watch any video online with Open-SUBTITLESFree...,1
1,1,2,"0.35028154,-0.11853625,-0.6772548,0.22706099,0...",9180533,the.message.(1976).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an...",Watch any video online with Open-SUBTITLESFree...,1
2,1,3,"-0.15815595,-0.15216613,-0.2958026,0.059707265...",9180533,the.message.(1976).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an...",Watch any video online with Open-SUBTITLESFree...,1
3,1,4,"-0.1082776,0.16064844,-0.11414268,0.40882686,0...",9180533,the.message.(1976).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an...",Watch any video online with Open-SUBTITLESFree...,1
4,2,1,"-0.40139484,0.18756877,0.42783958,0.018657252,...",9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther...",Ah! There's PrincessDawn and Terry with the2Bl...,2


In [14]:

merged_df = merged_df.drop('content',axis = 1)

In [15]:
##merged_df.to_csv(r"C:\Users\91998\OneDrive\Desktop\Internship_folder\Search_Engine\data\\final.csv",escapechar='\\')

In [17]:
import pandas as pd
df = pd.read_csv(r"C:\Users\91998\OneDrive\Desktop\Internship_folder\Search_Engine\data\final.csv")
df.head()

ParserError: Error tokenizing data. C error: out of memory